In [1]:
import numpy as np
import h5py
import math

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Conv2D, Conv3D
from tensorflow.keras.layers import Lambda, Concatenate, MaxPooling2D, Maximum, Add
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import Callback,LearningRateScheduler
from tensorflow.keras.utils import plot_model

import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [2]:
# Load Dataset from Google Drive
def load_train_dataset():
    """
    Load training dataset from Google Drive

    Returns:
    clean_image -- Clean image patches
    clean_image -- Hazy image patches
    transmission_value -- Transmission value which convert clean image to
                          haze image
    """

    file = 'dehaze.hdf5'
    train_dataset = h5py.File(file, 'r')
    clean_image = np.array(train_dataset['clear_image'][:]).astype(np.float32)
    haze_image = np.array(train_dataset['hazy_image'][:]).astype(np.float32)
    transmission_map_refine = np.array(train_dataset['transmission_map_refined']).astype(np.float32)
    train_dataset.close()

    return clean_image, haze_image, transmission_map_refine


# Gaussian Weight Initializtion for layers
weight_init = RandomNormal(mean=0.0, stddev=0.001)


# LearningRate Decay function
def lr_schedule(epoch,lr, logs={}):
    """
    Learning Rate Deacy scheduler

    Arguments:
    epoch -- current epoch number
    lr -- current learning rate
    log -- dictionary storing the logs of training

    Returns:
    lr -- learning rate for next epoch
    """

    print('learning_rate:',lr)
    logs.update({'lr': lr})
    if epoch in (49,99):
        return lr*0.5
    else:
        return lr

In [3]:
clean_image, haze_image, transmission_map_refine = load_train_dataset()

print ("Number of training examples:", clean_image.shape[0])
print ("Clean Image Patch shape:", clean_image.shape)
print ("Haze Image Patch shape:", haze_image.shape)
print ("Transmission Map shape:", haze_image.shape)
print ("Transmission Map Refine shape:", haze_image.shape)

Number of training examples: 956
Clean Image Patch shape: (956, 512, 512, 3)
Haze Image Patch shape: (956, 512, 512, 3)
Transmission Map shape: (956, 512, 512, 3)
Transmission Map Refine shape: (956, 512, 512, 3)


In [4]:
# residual_input = np.clip(((haze_image/255.0)/np.expand_dims(transmission_map_refine, axis=3)), 0, 1)
# residual_output = np.clip((residual_input - clean_image), 0, 1)

import numpy as np

# Define the batch size
batch_size = 32  # Adjust this based on your available memory

# Get the shape of the haze_image array
haze_shape = haze_image.shape

# Create an array to store the results
residual_input_result = np.empty(haze_shape, dtype=np.float32)
residual_output_result = np.empty(haze_shape, dtype=np.float32)

# Iterate over batches
for i in range(0, haze_shape[0], batch_size):
    # Define the current batch range
    batch_start = i
    batch_end = min(i + batch_size, haze_shape[0])

    # Extract the current batch
    current_batch_haze = haze_image[batch_start:batch_end]
    current_batch_transmission = transmission_map_refine[batch_start:batch_end]
    current_batch_clean = clean_image[batch_start:batch_end]

    # Perform operations on the current batch
    residual_input_batch = np.clip(((current_batch_haze / 255.0) / np.expand_dims(current_batch_transmission, axis=3)), 0, 1)
    residual_output_batch = np.clip((residual_input_batch - current_batch_clean), 0, 1)

    # Store the results in the corresponding positions
    residual_input_result[batch_start:batch_end] = residual_input_batch
    residual_output_result[batch_start:batch_end] = residual_output_batch

# The final results are stored in residual_input_result and residual_output_result


In [5]:
del clean_image, haze_image, transmission_map_refine

In [6]:
def ResidualBlock(X, iter):
    """
    Implementation of the single block of RNN

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    iter -- integer, used to name layers, depending on current residual block

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # Save the input value
    X_shortcut = X

    # BATCHNORMALIZATION -> CONV Block
    X = BatchNormalization(axis = 3, name = 'res_batchnorm_' + str(iter))(X)
    X = Conv2D(1, (3, 3), strides = (1,1), padding = 'same', kernel_initializer = weight_init, name = 'res_conv_' + str(iter))(X)

    # Add shortcut value to main path, and pass it through a RELU activation
    X = Add(name = 'res_add_'+ str(iter))([X,X_shortcut])
    X = Activation('relu', name = 'res_activation_'+ str(iter))(X)

    return X

def ResidualModel(input_shape):
    """
    Implementation of the Model.

    Arguments:
    input_shape -- shape of the images of the dataset
                   (height, width, channels) as a tuple.

    Returns:
    model -- a Model() instance in Keras
    """

    X_input = Input(input_shape, name = 'input1')

    # CONV -> RELU Block applied to X
    X = Conv2D(16, (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = weight_init, name = 'conv1')(X_input)
    X = Activation('relu', name = 'activation1')(X)

    # X = Conv2D(8, (1, 1), kernel_initializer = weight_init, name='test_conv')(X)

    for i in range(17):
        X = ResidualBlock(X, i)

    # CONV BLock
    X = Conv2D(3, (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = weight_init, name = 'conv2')(X)
    X = Activation('relu', name = 'activation2')(X)

    # Create Keras model instance
    model = Model(inputs = X_input, outputs = X, name='TransmissionModel')

    return model

In [7]:
model2 = ResidualModel(residual_input_result.shape[1:])
model2.summary()
model2.compile(optimizer=SGD(0.001), loss=MeanSquaredError())

Model: "TransmissionModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 512, 512, 16) 448         input1[0][0]                     
__________________________________________________________________________________________________
activation1 (Activation)        (None, 512, 512, 16) 0           conv1[0][0]                      
__________________________________________________________________________________________________
res_batchnorm_0 (BatchNormaliza (None, 512, 512, 16) 64          activation1[0][0]                
__________________________________________________________________________________

In [ ]:
history2 = model2.fit(residual_input_result, residual_output_result, batch_size = 30, epochs = 150, callbacks=[LearningRateScheduler(lr_schedule)])

In [7]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}